In [ ]:
# Python User-User Collaborative Filtering Recommender System
# consider not using pivot table?
# experiment with different similarity functions, number of neighbors, ...
# Predicted ratings could be rounded to increase accuracy

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('inputs/boardgame-elite-users.csv')
df = df.pivot_table(index='userID', columns='gameID', values='rating') 

df = df.fillna(0)

In [2]:
from sklearn.preprocessing import normalize

# Normalize Ratings for use in similarity derivation
normalized = normalize(df)

In [3]:
# Get User Similarity Matrix
# Also try Pearson coefficient, city-block, etc...
from sklearn.metrics.pairwise import cosine_similarity

sim = cosine_similarity(normalized)

In [4]:
# Get indices of most similar users, argsort returns list of indices of the values
# as if the values had been sorted in descending order
nbor_indx = sim.argsort()[:,::-1]

# Helper function called by "get_ten_nearest"
# Takes a gameID and returns list of userID
def get_users_that_rated(gid):
    return df[gid].where(df[gid]>0).dropna().index.tolist()
    
# Helper function called by "predict_rating"
# Takes df, userID, and gameID that has not been rated by userID ; returns list of 10 most similar userIDs 
# THAT HAVE rated the game (descending order of similarity)
def get_k_nearest_neighbors(uid, gameid, k):
    assert(k>1)
    pot_nbors = get_users_that_rated(gameid)
    uid_indx = df.index.get_loc(uid)
    
    sim_userIDs = []
    for x in nbor_indx[uid_indx]:
        if df.index[x] in pot_nbors:
            sim_userIDs.append(df.index[x])
        if len(sim_userIDs)>k: # self will be the first member(total 11)
            break
    return sim_userIDs[1:]
    
print(get_k_nearest_neighbors(66705,13, 10))

[187094, 128117, 55092, 13340, 19677, 37860, 12305, 96817, 119312, 27853]


In [21]:
print(df.index)
print(df.columns)

Int64Index([   272,    388,    430,   2044,   3080,   3256,   3557,   5038,
              5217,   5480,
            ...
            180167, 180775, 181123, 181339, 181472, 187094, 189973, 191116,
            192057, 193339],
           dtype='int64', name='userID', length=199)
Int64Index([     3,      5,     10,     11,     12,     13,     18,     41,
                42,     45,
            ...
            187645, 192291, 193738, 194655, 198773, 201808, 204583, 205059,
            205637, 209010],
           dtype='int64', name='gameID', length=402)


In [5]:
# Takes df, userID, gameID
# returns predicted rating for gameID
def predict_rating(uid, gameid, k=10):
    sim_uids = get_k_nearest_neighbors(uid, gameid, k)
    accm_score = 0.0
    accm_weight = 0.0
    for nbor_id in sim_uids:
        sim_indx_x = df.index.get_loc(uid)
        sim_indx_y = df.index.get_loc(nbor_id)
        accm_score += (df.loc[nbor_id][gameid] * sim[sim_indx_x][sim_indx_y])
        accm_weight += sim[sim_indx_x][sim_indx_y]
    score = accm_score / accm_weight
    return score

predict_rating(3080, 187645)

7.613137767277404

In [25]:
# Need to use apply where each rating is zero
# This really doesn't make sense, as predict_rating doesn't translate the dataframe value into something
# Probably can't use 'apply' in the form things are in...

pred_mat = df.where(df==0).apply(lambda x: predict_rating(x.index, x.name))

TypeError: ("'Int64Index([   272,    388,    430,   2044,   3080,   3256,   3557,   5038,\n              5217,   5480,\n            ...\n            180167, 180775, 181123, 181339, 181472, 187094, 189973, 191116,\n            192057, 193339],\n           dtype='int64', name='userID', length=199)' is an invalid key", 'occurred at index 3')